# Assignment 2: MongoDB and Flask
 


Objective: to demonstrate how we can create a simple yet usable data product using MongoDB and data from different data sources.

Step 1: Using same books.csv file from Assignment 1, in a Jupyter notebook and 
Cleaning date data rows (6/31 and 11/31)

In [1]:
import pandas as pd #import pandas Library

In [2]:
df = pd.read_csv("books-5.csv")
#Convert the 'date' column to a pandas datetime format
df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')

#Drop any rows with Na values
df = df.dropna()

In [3]:
df

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1.0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,9.780000e+12,eng,652.0,2095690.0,27591.0,2006-09-16,Scholastic Inc.
1,2.0,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,9.780000e+12,eng,870.0,2153167.0,29221.0,2004-09-01,Scholastic Inc.
2,4.0,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,9.780000e+12,eng,352.0,6333.0,244.0,2003-11-01,Scholastic
3,5.0,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9.780000e+12,eng,435.0,2339585.0,36325.0,2004-05-01,Scholastic Inc.
4,8.0,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,439682584,9.780000e+12,eng,2690.0,41428.0,164.0,2004-09-13,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11122,45631.0,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9.780000e+12,eng,512.0,156.0,20.0,2004-12-21,Da Capo Press
11123,45633.0,You Bright and Risen Angels,William T. Vollmann,4.08,140110879,9.780000e+12,eng,635.0,783.0,56.0,1988-12-01,Penguin Books
11124,45634.0,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,140131965,9.780000e+12,eng,415.0,820.0,95.0,1993-08-01,Penguin Books
11125,45639.0,Poor People,William T. Vollmann,3.72,60878827,9.780000e+12,eng,434.0,769.0,139.0,2007-02-27,Ecco


Step 2: Creating MongoDB db and collection
Creating/inserting documents from file to db

*Handling inconsistencies in data and the presence of multiple authors*: 

In document DBs like MongoDB, avoiding duplicate data isn't as important as in a relational model because data can be easily nested and denormalized. This means that there is more flexibility in how data is structured, and it can be optimized for specific queries. 

In the following code, multiple authors are handled by splitting the authors column by commas and inserting each author as a separate document in the authors collection. Then, for each book, it associates multiple authors with the book in the books_authors collection by inserting a new document for each author-book pair.


*Consider: Given the data product we’re looking to make? Does it make sense to store the data in an author-centric fashion or book-centric? Or both?*

Since we are searching for specific authors and adding their works and bio info to their existing documents in the database/collection, it makes sense to have the data stored in an author-centric or hybrid fashion. This way, we can easily update the author document with the new information without having to search through all the books in the collection.

In [4]:
import pymongo

In [5]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client["booksdb"] #creating the "booksdb" database 

In [6]:
#creating the "books", "publishers", "languages", "books_authors", and "authors" collections
books = db["books"]
publishers = db["publishers"]
languages = db["languages"]
books_authors = db["books_authors"]
authors = db["authors"]

In [7]:
#inserting data into the "publishers" collection
publisher_ids = {}
for i, row in df[["publisher"]].iterrows():
    publisher_name = row["publisher"]
    if publisher_name not in publisher_ids:
        publisher_ids[publisher_name] = publishers.insert_one({"publisher_name": publisher_name}).inserted_id

In [8]:
#inserting data into the "languages" collection
language_ids = {}
for i, row in df[["language_code"]].iterrows():
    language_code = row["language_code"]
    if language_code not in language_ids:
        language_ids[language_code] = languages.insert_one({"language_code": language_code}).inserted_id

In [9]:
#inserting data into the "books" collection
for i, row in df.iterrows():
    book_doc = {
        "bookID": row["bookID"],
        "title": row["title"],
        "num_pages": row["  num_pages"],
        "isbn": row["isbn"],
        "publication_date": row["publication_date"],
        "ratings_count": row["ratings_count"],
        "text_reviews_count": row["text_reviews_count"],
        "isbn13": row["isbn13"],
        "average_rating": row["average_rating"],
        "publisher_id": publisher_ids[row["publisher"]],
        "language_id": language_ids[row["language_code"]]
    }
    books.insert_one(book_doc)

In [10]:
#inserting data into the "authors" collection and the "books_authors" collection
author_ids = {}
for i, row in df[["bookID", "authors"]].iterrows():
    if not pd.isna(row["authors"]):
        author_names = row["authors"].split(",")
        for author_name in author_names:
            author_name = author_name.strip()
            if author_name not in author_ids:
                author_ids[author_name] = authors.insert_one({"author_name": author_name}).inserted_id

        book_id = row["bookID"]
        for author_name in author_names:
            author_name = author_name.strip()
            author_id = author_ids[author_name]
            books_authors.insert_one({"book_id": book_id, "author_id": author_id})

Step 3:

Using OpenLibrary API…
Conduct a search in your MongoDB instance for the following 3 authors:
JK Rowling
Stephen King
Gabriel Garcia Marquez
Using each author name, conduct a search on https://openlibrary.org for the author’s JSON document:
To find author (example: https://openlibrary.org/search/authors.json?q=firstname%20lastname
In the resulting doc(s), find your author’s key
Use that key (for example: OL123456) to find their works and bio info: https://openlibrary.org/authors/OL123456.json
Add this information to their existing document(s) in your MongoDB database/collection.

In [11]:
import requests
import json


#defining list of authors to search for
authors = ['JK Rowling', 'Stephen King', 'Gabriel Garcia Marquez']

#looping through authors and fetch their data from OpenLibrary
for author in authors:
    #searching for author's JSON document on OpenLibrary
    url = f'https://openlibrary.org/search/authors.json?q={author.replace(" ", "%20")}'
    response = requests.get(url)
    data = response.json()
    
    #checking if author was found
    if data['numFound'] > 0:
        #fetching author's key from JSON document
        author_key = data['docs'][0]['key'].replace('/authors/', '') #using only the first key from the document
        
        #fetching author's works and bio info from OpenLibrary
        url = f'https://openlibrary.org/authors/{author_key}.json'
        response = requests.get(url)
        data = response.json()
        
        #print author's name and JSON document
        print(f"Author: {author}")
        print(data)
        print("\n")
        filename = f'{author}.json'
        with open(filename, 'w') as f:
            json.dump(data, f)


Author: JK Rowling
{'wikipedia': 'http://en.wikipedia.org/wiki/J._K._Rowling', 'personal_name': 'J. K. Rowling', 'key': '/authors/OL23919A', 'alternate_names': ['Joanne Rowling', 'Joanne K. Rowling', 'Jo Murray', 'Kennilworthy Whisp', 'JK Rowling', 'Robert Galbraith', 'Джоан Роулінг', 'Dzhoan Rouling', 'J.K. Rowling', 'ROWLING', 'روبرت غالبريث - Robert Galbraith', 'Rouling Dzh.K.', 'Newt Scamander', 'Newton Scamander'], 'remote_ids': {'wikidata': 'Q34660', 'isni': '000000012148628X', 'goodreads': '1077326', 'viaf': '116796842', 'librarything': 'rowlingjk', 'amazon': 'B000AP9A6K'}, 'type': {'key': '/type/author'}, 'links': [{'title': 'Official Site', 'url': 'http://www.jkrowling.com/', 'type': {'key': '/type/link'}}], 'name': 'J. K. Rowling', 'title': 'OBE', 'birth_date': '31 July 1965', 'entity_type': 'person', 'photos': [5543033, -1], 'source_records': ['amazon:8893817055', 'amazon:4863895666', 'amazon:1526618249', 'amazon:8862561512', 'amazon:1526618230', 'amazon:6073193009', 'amazon

In [12]:
#defining query and searching for the instance in mongodb collection "authors"
query = {"author_name": {"$in": ["JK Rowling", "Stephen King", "Gabriel Garcia Marquez"]}}
new_value = {"$set":{"Source_records":data["source_records"],"Bio":data['bio']}}
#inserting the new information to the author collection
db.authors.update_many(query, new_value)

In [13]:
cursor = db.authors.find(query)
for document in cursor:
    print(document)


{'_id': ObjectId('6408ef665451140722db58d6'), 'author_name': 'Stephen King', 'Bio': 'Gabriel García Márquez is a Colombian novelist, short-story writer, screenwriter and journalist. García Márquez, affectionately known as "Gabo" throughout Latin America, is considered one of the most significant authors of the 20th century. In 1982, he was awarded the Nobel Prize in Literature. He pursued a self-directed education that resulted in his leaving law school for a career in journalism. From early on, he showed no inhibitions in his criticism of Colombian and foreign politics. In 1958, he married Mercedes Barcha; they have two sons, Rodrigo and Gonzalo.\r\n\r\nHe started as a journalist, and has written many acclaimed non-fiction works and short stories, but is best-known for his novels, such as One Hundred Years of Solitude (1967) and Love in the Time of Cholera (1985). His works have achieved significant critical acclaim and widespread commercial success, most notably for popularizing a li

Step 4: 
    
Using Python Flask, create:
A simple home page with 3 links, one for each author’s bio page
a single, simple bio page for each author with biographical information as well as links to each of their books on Amazon (if Amazon ID exists)

Consider: do you need a hard-coded HTML file for each author, or can you leverage Flask and GET or POST requests? 
Since a dynamic webpage is not required here, in my opinion, a hard-coded HTML file may be the better choice. This approach is simpler to implement and has faster load times compared to generating the HTML dynamically with Flask. 

In [ ]:
from flask import Flask, render_template
import json

app = Flask(__name__)

def load_data(file_name):
    with open(file_name) as f:
        data = json.load(f)
    source_records = data['source_records']
    bio = data['bio']
    books = []
    for record in source_records:
        if record.startswith('amazon:'):
            books.append({
                'amazon_id': record.split(':')[1]
        })
    return bio, books


@app.route('/')
def home():
    return render_template('homepage.html')

@app.route('/author/<author_name>')
def author(author_name):
    # Load author's data from JSON file
    file_name = author_name.lower().replace(' ', '-') + '.json'
    bio, books = load_data(file_name)
    # Render author's bio page template and pass in author's data
    return render_template('home.html', author_name1 = author_name.title().replace('-', ' '), bio=bio, books=books)

if __name__ == '__main__':
    app.run(port=4995)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4995
Press CTRL+C to quit
127.0.0.1 - - [15/Mar/2023 19:56:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2023 19:56:33] "GET /static/rawkkim-RYGrvZWSMNE-unsplash.jpg HTTP/1.1" 304 -
127.0.0.1 - - [15/Mar/2023 19:56:33] "GET /static/rawkkim-RYGrvZWSMNE-unsplash.jpg HTTP/1.1" 304 -
127.0.0.1 - - [15/Mar/2023 19:56:34] "GET /author/gabriel-garcia-marquez HTTP/1.1" 200 -
127.0.0.1 - - [15/Mar/2023 19:56:34] "GET /static/authors/author_name.jpg HTTP/1.1" 404 -
